## RCV1-v2 Dataset [source here](http://www.jmlr.org/papers/volume5/lewis04a/lyrl2004_rcv1v2_README.htm)

In [1]:
import logging

from sklearn.datasets import fetch_rcv1
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn import svm
from keras.layers import Input, Dense
from keras.models import Model

logging.basicConfig()
rcv1 = fetch_rcv1()

Using Theano backend.


In [13]:
training_samples = 23149

X_train = rcv1.data[:training_samples].todense()
# X_test = rcv1.data[training_samples:].todense()

# X_train_rshp = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
# X_test_rshp = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))
# y_train = rcv1.target[:training_samples]
# y_test = rcv1.target[training_samples:]

X_train.shape

# X_train.shape, X_train_rshp.shape,  X_test.shape, X_test_rshp.shape, y_train.shape, y_test.shape

MemoryError: 

In [7]:
encoding_dim = 32

input_doc = Input(shape=(23149,))

encoded = Dense(encoding_dim,activation='relu')(input_doc)
decoded = Dense(23149, activation='sigmoid')(encoded)

autoencoder = Model(input=input_doc, output=decoded)

encoder = Model(input=input_doc, output=encoded)

encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(X_train, X_train,
                verbose=2,
                nb_epoch=50,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))


Exception: Error when checking model input: expected input_2 to have shape (None, 23149) but got array with shape (23149, 47236)

In [3]:
clf = OneVsRestClassifier(svm.LinearSVC(penalty='l1',tol=0.01,multi_class='crammer_singer',dual=False))

In [4]:
clf.fit(X_train,y_train)

/home/felipe/venv2/local/lib/python2.7/site-packages/sklearn/multiclass.py:70: UserWarning: Label not 49 is present in all training examples.
  str(classes[c]))
/home/felipe/venv2/local/lib/python2.7/site-packages/sklearn/multiclass.py:70: UserWarning: Label not 80 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l1', random_state=None,
     tol=0.01, verbose=0),
          n_jobs=1)

In [5]:
y_pred = clf.predict(X_test)

current_score = f1_score(y_test,y_pred,average='micro')

In [6]:
current_score

0.80843419139591599